In [ ]:
# 从SQuAD 2.0出去文本数据，加入随机噪音生成训练集。
# 比较不同的model的纠错效果。
# 英文数据

import json
import tensorflow as tf
import math
import random
import numpy as np
from numpy import zeros, newaxis
from numpy.random import choice as random_choice, randint as random_randint, shuffle as random_shuffle, seed as random_seed
import pandas as pd
import sys
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, LSTM, Embedding
from keras.preprocessing.text import Tokenizer

english_text_file = "./input_data/abstract_english_text_file"
clean_english_text_file = "./input_data/clean_english_text_file"
before_add_error_file = "./input_data/before_add_error"
after_add_error_file = "./input_data/after_add_error"
change_index_file = "./input_data/change_index_file"

err_prob = {
    "replace_one_char": 0.4,
    "add_one_char": 0.2,
    "delete_one_char": 0.2,
    "change_neighbor_order": 0.2
}

# max_error_rate = 0.2
char_list = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .")
max_error_line_length_rate = 0.05

# 函数用来parse SQuAD 2.0的数据
# 我们只需要抽取文本信息, SQuAD2.0的数据格式比较奇怪
def read_squad_examples(input_file, is_training):
    with tf.io.gfile.GFile(input_file, "r") as reader:
        input_data = json.load(reader)["data"]
    return input_data

def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

# 抽取英文文本，中文和其他类别的去掉
def abstarct_sentence(json_data):
    global english_text_file
    fp = open(english_text_file, "w")
    for article in json_data:
        paragraphs = article["paragraphs"]
        for paragraph in paragraphs:
            qas = paragraph["qas"]
            for qa in qas:
                question = qa["question"]
                if isEnglish(question):
                    fp.write(question + "\n")
                answer_struct = qa["answers"]
                if (len(answer_struct) > 0):
                    answer = answer_struct[0]["text"]
                    if isEnglish(answer):
                        fp.write(answer + "\n")
    fp.close()
    print("abstract finish")

def clean_file(file=english_text_file):
    global clean_english_text_file
    fp = open(file, "r")
    fp2 = open(clean_english_text_file, "w")
    for line in fp:
        line = line.strip()
        if line != "" and line != "null" and len(line) > 10:
            fp2.write(line + "\n")
    fp.close()
    fp2.close()

def map_prob_to_range_with_keys(key_prob):
    prob_list = list(key_prob.values())
    prob_sum = sum(map(float,prob_list))
    key_range = dict()
    if math.isclose(prob_sum, 1) == False:
        print("prob sum is not 1")
        sys.exit(-1)
    else:
        threshold = 0
        for key, prob in key_prob.items():
            key_range[key] = [threshold, threshold + prob]
            threshold += prob
    return key_range

def choose_item_based_on_prob(key_range):
    value = random.uniform(0, 1)
    last_key = None
    for key, prob_range in key_range.items():
        last_key = key
        if value >= prob_range[0] and value < prob_range[1]:
            return key
    return last_key

# 随机修改正确的句子到错误的句子
# 包括添加字符，删除字符，交换临近字符，替换字符
def add_error_to_line(line):
    max_error_line_length_rate
    max_error_num = (int)(max_error_line_length_rate * len(line))
    set_error_num = (int)(random.uniform(0, 1) * max_error_num)
    cur_error_num = 0
    before = line
    after = line
    key_range = map_prob_to_range_with_keys(err_prob)
    while cur_error_num < set_error_num:
        err_type = choose_item_based_on_prob(key_range)
        cur_error_num += 1
        if err_type == "replace_one_char":
            random_char_position = random_randint(len(after))
            after = after[:random_char_position] + random_choice(char_list[:-1]) \
                    + after[random_char_position + 1:]
        elif err_type == "add_one_char":
            random_char_position = random_randint(len(after))
            after = after[:random_char_position] + random_choice(char_list[:-1]) \
                    + after[random_char_position:]
        elif err_type == "delete_one_char":
            random_char_position = random_randint(len(after))
            after = after[:random_char_position] + after[random_char_position + 1:]
        elif err_type == "change_neighbor_order":
            random_char_position = random_randint(len(after) - 1)
            after = (after[:random_char_position] + after[random_char_position + 1] \
                     + after[random_char_position] + after[random_char_position + 2:])
    return before, after


def gen_X_y(tk):
    global clean_english_text_file, before_add_error_file, after_add_error_file
    X = []
    Y = []
    X_encoding = []
    Y_encoding = []
    before_file = open(before_add_error_file, "w")
    after_file = open(after_add_error_file, "w")
    change_index_fp = open(change_index_file, "w")
    ori_file = open(clean_english_text_file, "r")
    line_num = 1
    for line in ori_file:
        line = line.strip("\n")
        correct, mistaken = add_error_to_line(line)
        X.append([mistaken])
        Y.append([correct])
        before_file.write(correct + "\n")
        after_file.write(mistaken + "\n")
        if correct != mistaken:
            change_index_fp.write(str(line_num) + "\n")
        line_num += 1
        X_encoding.append(tk.texts_to_sequences([mistaken])[0])
        Y_encoding.append(tk.texts_to_sequences([correct])[0])
#     np_array_X = np.array(X_encoding)
#     np_array_Y = np.array(Y_encoding)
    np_array_X = np.array([np.array(xi) for xi in X_encoding])
    np_array_Y = np.array([np.array(yi) for yi in Y_encoding])
    return np_array_X, np_array_Y

# DL要求将字符转成int作为输入
def char2int(file=clean_english_text_file):
    fp = open(file, "r")
    all_text = fp.read()
    fp.close()
    tk = Tokenizer(num_words=None, char_level=True, oov_token="UNK", lower=False)
    tk.fit_on_texts(all_text)
    return tk



In [ ]:
# 1. 抽取SQuAD 2.0的问答数据
json_data = read_squad_examples("./input_data/train-v2.0.json", False)
abstarct_sentence(json_data)
clean_file()

# 2. 生成码表
tk = char2int()
print(tk.word_index)
char_size = len(tk.word_index) + 1
print(char_size)

# 3. 在抽取的文件中加入错误的噪音
np_array_X, np_array_Y = gen_X_y(tk)
# 确保行数一样
print(np_array_X[0:5])
print(np_array_Y[0:5])
print(type(np_array_X))
print(type(np_array_Y))
print(np_array_X.shape)
print(np_array_Y.shape)

# 4. Y转化为one-hot
rows = np_array_Y.shape[0]
one_hot_Y = []
for x in range(0, rows):
    cur_one_hot = to_categorical(np_array_Y[x], num_classes=char_size)
    one_hot_Y.append(cur_one_hot)


In [ ]:
print(np_array_X[0:5])
print(one_hot_Y[0:5])
print(char_size)
print(len(np_array_X[0]))
print(len(one_hot_Y[0]))

In [ ]:
# 这个是many to many，应该用encoder-decoder方法
def simple_char_model(char_size):
    print('Build model...')
    model = Sequential()
    # 输入序列变长，timestep设置为None.
    # LSTM作为首层才需要设置input_shape参数。
    model.add(LSTM(100, input_shape=(None, char_size), return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(Dense(char_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

print(np_array_X.shape)
# np_array_X_3d = np_array_X[..., newaxis]
# print(np_array_X_3d.shape)
# print(np_array_X_3d[0:5])
# 输入不对，是不是要把每个sample的输入变成[[1], [2], [3]]，而不是[1,2,3]? 这样的input_shape怎么设？是(None,1)还是(None, char_size)
model1 = simple_char_model(char_size)
model1.fit(np_array_X, one_hot_Y)
